In [1]:
import numpy as np
from scipy.integrate import odeint
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Slider, CustomJS
from bokeh.layouts import column, row

output_notebook()

N = 10000
I0 = 5
R0 = 1
S0 = N - I0 - R0
tmax = 100
Nt = 200
t = np.linspace(0, tmax, Nt+1)

def SIR(X, t, beta, gamma):
    S, I, R = X
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return np.array([dSdt, dIdt, dRdt])

def update_data(beta, gamma):
    y0 = S0, I0, R0
    result = odeint(SIR, y0, t, args=(beta, gamma))
    S, I, R = result.T
    return t, S, I, R

beta, gamma = 0.3, 0.1
t, S, I, R = update_data(beta, gamma)

data = ColumnDataSource(data={
    't': t,
    'S': S,
    'I': I,
    'R': R
})

plot = figure(title="Model SIR", x_axis_label='Czas', y_axis_label='Populacja', width=800, height=400)
plot.line('t', 'S', source=data, color="blue", legend_label="Podatni (S)")
plot.line('t', 'I', source=data, color="red", legend_label="Zainfekowani (I)")
plot.line('t', 'R', source=data, color="green", legend_label="Ozdrowiali (R)")
plot.legend.location = "right"

beta_slider = Slider(start=0.1, end=1.0, value=0.3, step=0.01, title="Beta")
gamma_slider = Slider(start=0.05, end=0.5, value=0.1, step=0.01, title="Gamma")

callback = CustomJS(args=dict(source=data, beta_slider=beta_slider, gamma_slider=gamma_slider, t=t, N=N, S0=S0, I0=I0, R0=R0), code="""
    const beta = beta_slider.value;
    const gamma = gamma_slider.value;
    const y0 = [S0, I0, R0];
    const Nt = t.length;
    const S = new Array(Nt).fill(0);
    const I = new Array(Nt).fill(0);
    const R = new Array(Nt).fill(0);

    S[0] = y0[0];
    I[0] = y0[1];
    R[0] = y0[2];

    for (let i = 1; i < Nt; i++) {
        const dSdt = -beta * S[i-1] * I[i-1] / N;
        const dIdt = beta * S[i-1] * I[i-1] / N - gamma * I[i-1];
        const dRdt = gamma * I[i-1];

        S[i] = S[i-1] + dSdt * (t[i] - t[i-1]);
        I[i] = I[i-1] + dIdt * (t[i] - t[i-1]);
        R[i] = R[i-1] + dRdt * (t[i] - t[i-1]);
    }

    source.data = { t: t, S: S, I: I, R: R };
    source.change.emit();
""")

beta_slider.js_on_change('value', callback)
gamma_slider.js_on_change('value', callback)

layout = column(plot, row(beta_slider, gamma_slider))

show(layout)


Loading BokehJS ...